## Лабораторна робота 7

1. Створити кілька своїх прикладів у форматі json за тематикою варіанту
(англійською або українською мовою) для розпізнавання нового типу
сутностей (обрати самостійно). Створити програму, що додає ці
приклади до існуючої моделі spaCy, навчає модель. Продемонструвати
роботу.

2. Застосувати компонент TextCategorizer для визначення намірів. Дані для
навчання за тематикою варіанту обрати самостійно або скористатись
вказаним файлом (utterance містить висловлювання, intent - намір). Дані
файли містять приклади діалогів користувачів з системою-помічником за
певною тематикою, наприклад, замовлення квитків і т.д. Навчити
компонент та продемонструвати роботу.

##### 1. Додавання нових прикладів розпізнавання в уже існуючу модель spaCy

In [96]:
import spacy
from spacy.training import Example, offsets_to_biluo_tags
import random

In [97]:
# Завантажуємо модель spaCy
nlp = spacy.load("en_core_web_md")

In [115]:
# Cтворюємо нові приклади з категорією номер рахунку
train_data = [
    ("Please transfer 87893789 bucks to account number 12345678.", {"entities": [(49, 57, "ACCOUNT_NUMBER")]}),
    ("The account number for the transfer is 87654321.", {"entities": [(39, 47, "ACCOUNT_NUMBER")]}),
    ("Send the money to 11223344 immediately.", {"entities": [(18, 26, "ACCOUNT_NUMBER")]}),
]

In [116]:
# Перевірка правильності індексів сутностей
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    entities = annotations["entities"]
    tags = offsets_to_biluo_tags(doc, entities)
    
    print(f"Text: {text}")
    print(f"Entities: {entities}")
    print(f"BILUO Tags: {tags}\n")

Text: Please transfer 87893789 bucks to account number 12345678.
Entities: [(49, 57, 'ACCOUNT_NUMBER')]
BILUO Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-ACCOUNT_NUMBER', 'O']

Text: The account number for the transfer is 87654321.
Entities: [(39, 47, 'ACCOUNT_NUMBER')]
BILUO Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-ACCOUNT_NUMBER', 'O']

Text: Send the money to 11223344 immediately.
Entities: [(18, 26, 'ACCOUNT_NUMBER')]
BILUO Tags: ['O', 'O', 'O', 'O', 'U-ACCOUNT_NUMBER', 'O', 'O']



In [100]:
# Додаємо новий тег для розпізнавання
ner = nlp.get_pipe("ner")
ner.add_label("ACCOUNT_NUMBER")

1

In [101]:
# Готуємо приклади
train_examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in train_data]

In [102]:
# Навчання моделі
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
epoches = 20

# Вимикаємо усі компоненти
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()
    for epoch in range(epoches): 
        random.shuffle(train_examples)
        for example in train_examples:
            nlp.update([example], sgd=optimizer)


In [103]:
# Зберігаємо нову модель
ner.to_disk("new_ner_model")

In [104]:
# Тестуємо модель
test_text = ["Transfer 1000 bucks to the account 98765432.",
             "Make it one thousand one hundred and sixty bucks, but it's going to 11235432.",
             "Transfer 98765432 bucks to the account 1111111.",]

for text in test_text:
    doc = nlp(text)
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('98765432', 'ACCOUNT_NUMBER')]
Entities: [('11235432', 'ACCOUNT_NUMBER')]
Entities: [('1111111', 'ACCOUNT_NUMBER')]


##### 2.Використання TextCategorizer для визначення намірів

In [105]:
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL

In [106]:
# Завантаження існуючої моделі spaCy
nlp = spacy.load("en_core_web_md")

In [107]:
# Приклад даних з файлу banks.json
train_data = [
    # TransferMoney
    ("I want to send $200 to my friend.", {"cats": {"TransferMoney": 1.0}}),
    ("Please transfer $1000 to John's account.", {"cats": {"TransferMoney": 1.0}}),
    ("Transfer $300 to my savings account.", {"cats": {"TransferMoney": 1.0}}),
    
    # CheckBalance
    ("Can you check my account balance?", {"cats": {"CheckBalance": 1.0}}),
    ("What is the balance in my savings account?", {"cats": {"CheckBalance": 1.0}}),
    ("Tell me my current balance.", {"cats": {"CheckBalance": 1.0}}),
    
    # ConfirmTransfer
    ("Yes, I confirm the transfer of $500.", {"cats": {"ConfirmTransfer": 1.0}}),
    ("I authorize the transfer of $300 to my friend.", {"cats": {"ConfirmTransfer": 1.0}}),
    ("Confirm that I want to send $200 to John's account.", {"cats": {"ConfirmTransfer": 1.0}})
]

In [108]:
# Додавання TextCategorizer до конвеєра обробки природньої мови
config = {
    "threshold": 0.5,
    "model": DEFAULT_SINGLE_TEXTCAT_MODEL  
}

textcat = nlp.add_pipe("textcat", config=config)

In [109]:
# Додавання міток категорій (намірів)
textcat.add_label("TransferMoney")
textcat.add_label("CheckBalance")
textcat.add_label("ConfirmTransfer")

1

In [110]:
# Підготовка навчальних прикладів
train_examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in train_data]

# Ініціалізація компонента TextCategorizer
textcat.initialize(lambda: train_examples, nlp=nlp)

# Навчання моделі
epochs = 20
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.resume_training()
    for epoch in range(epochs):
        random.shuffle(train_examples)
        for example in train_examples:
            nlp.update([example], sgd=optimizer)


In [111]:
# Збереження навченої моделі
nlp.to_disk("intent_model")

In [112]:
# Завантаження навченої моделі для тестування
my_model = spacy.load("intent_model")

# Тестування моделі
test_text = ["I would like to transfer $200 to my savings account.",
             "Transfer $400 from my checking account.",
             
            "Could you please check my account balance?", 
            "Your savings is sitting at $11,339.61 right now",
             
            "Yes, I authorize the transfer of $300 to my friend."]


for text in test_text:
    doc = my_model(text)
    print("Text:", text)
    print("Categories:", doc.cats, end="\n\n")

Text: I would like to transfer $200 to my savings account.
Categories: {'TransferMoney': 0.9945172667503357, 'CheckBalance': 0.0017630599904805422, 'ConfirmTransfer': 0.0037196753546595573}

Text: Transfer $400 from my checking account.
Categories: {'TransferMoney': 0.9405916929244995, 'CheckBalance': 0.04289665073156357, 'ConfirmTransfer': 0.01651166006922722}

Text: Could you please check my account balance?
Categories: {'TransferMoney': 0.0006578673492185771, 'CheckBalance': 0.9988841414451599, 'ConfirmTransfer': 0.0004579682135954499}

Text: Your savings is sitting at $11,339.61 right now
Categories: {'TransferMoney': 0.07760903239250183, 'CheckBalance': 0.5011147856712341, 'ConfirmTransfer': 0.4212762415409088}

Text: Yes, I authorize the transfer of $300 to my friend.
Categories: {'TransferMoney': 0.0002088824548991397, 'CheckBalance': 0.00019267962488811463, 'ConfirmTransfer': 0.9995984435081482}

